In [1]:
import os
import workspace_path
os.chdir(workspace_path.path)
os.getcwd()

'e:\\Code\\openmmlab\\mmsegmentation'

## 载入ONNX模型

In [2]:
onnx_seg = '../mmdeploy/mmseg2onnx_deeplabv3plus'  # 这里载入的是生成的文件夹, end2end.onnx在里面

from mmdeploy_runtime import Segmentor
device = 'cpu'
# device = 'cuda'   # 当使用cuda时, 由于模型推理占用了内核, 所以无法在notebook中执行代码, 需要在终端本地运行
segmentor = Segmentor(onnx_seg, device)

## 颜色配置

In [3]:
palette = [
    ['background', [127,127,127]],
    ['red', [0,0,200]],
    ['green', [0,200,0]],
    ['white', [144,238,144]],
    ['seed-black', [30,30,30]],
    ['seed-white', [8,189,251]]
]

palette_dict = {}
for idx, each in enumerate(palette):
    palette_dict[idx] = each[1]

## 单帧处理函数

In [4]:
import cv2
import numpy as np

def process_frame(img_bgr, opacity=0.3):
    
    # 语义分割推理预测
    pred_mask = segmentor(img_bgr)
    # 将预测的整数ID，映射为对应类别的颜色
    pred_mask_bgr = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3))
    for idx in palette_dict.keys():
        pred_mask_bgr[np.where(pred_mask==idx)] = palette_dict[idx]
    pred_mask_bgr = pred_mask_bgr.astype('uint8')
    # 将语义分割预测图和原图叠加显示
    img_bgr = cv2.addWeighted(img_bgr, opacity, pred_mask_bgr, 1-opacity, 0)
    return img_bgr

## 视频逐帧处理

In [5]:
from tqdm import tqdm
def generate_video(input_video_path, output_path):
    print('视频开始处理',input_video_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_video_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    # cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_video_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                # frame_path = './temp_frame.png'
                # cv2.imwrite(frame_path, frame)
                try:
                    frame = process_frame(frame)
                except:
                    print('报错！error')
                    pass
                
                if success == True:
                    # cv2.imshow('Video Processing', frame)
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

## 视频预测

In [6]:
generate_video(input_video_path='data/video_watermelon_2.mp4', output_path='myFolder/outputs/video_watermelon_2_onnx.mp4')

视频开始处理 data/video_watermelon_2.mp4
视频总帧数为 260


100%|██████████| 259/259 [55:59<00:00, 12.97s/it]

视频已保存 myFolder/outputs/video_watermelon_2_onnx.mp4
